In [12]:
!pip install gspread pandas oauth2client

In [13]:
import os
import json
import gspread
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials

# Ưu tiên lấy từ biến môi trường
if "SERVICE_ACCOUNT_JSON" in os.environ:
    service_account_info = json.loads(os.environ["SERVICE_ACCOUNT_JSON"])
else:
    # Fallback: đọc từ file service_account.json (bạn phải upload lên Colab hoặc để sẵn trong project)
    with open("service_account.json") as f:
        service_account_info = json.load(f)

# Authenticate Google Sheets API
scope = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/drive"
]
creds = ServiceAccountCredentials.from_json_keyfile_dict(service_account_info, scope)
client = gspread.authorize(creds)

# Mở Google Sheet
spreadsheet = client.open_by_url(
    "https://docs.google.com/spreadsheets/d/1RNrJ5B3pzLGNmxir2EguHH55vGUNHAOzaOV6Ng6_fVI/edit?gid=843162208#gid=843162208"
)
sheet = spreadsheet.worksheet("Jobs 2025")
data = sheet.get_all_records()

# Convert to DataFrame
df = pd.DataFrame(data)
print(df.head())


   0       Id                                      Title  \
0  1   454474                           Thợ Hàn Mig, Tig   
1  2  1060402              Thiết Kế Kiến Trúc - Nội Thất   
2  3  1128390  Nhân Viên Kỹ Thuật - Cơ Điện - Tại Quận 5   
3  4  1267000                        Nhân Viên Marketing   
4  5  1305600   Nhân Viên Xử Lý Khiếu Nại - Dự Án Shopee   

                         Company                 Category       Places  \
0  VU TRINH CNC GROUP CO., LTD\n  Cơ Khí/Ô Tô/Tự Động Hóa  Hồ Chí Minh   
1                 Kiến Trúc Yama       Kiến Trúc/Nội Thất  Hồ Chí Minh   
2               TPV TECH CO.,LTD        Khoa Học/Kỹ Thuật  Hồ Chí Minh   
3                    greendragon      Kinh Doanh/Bán Hàng  Hồ Chí Minh   
4  VIET MY - SSU COMPANY LIMITED      Chăm Sóc Khách Hàng  Hồ Chí Minh   

    Districts  Difficulty                 Sale     Package  ...  Views  \
0  Bình Chánh         1.0  Võ Thị Hồng Trâm    Standard  ...    175   
1                     1.0      Trịnh Thị Tuyến

In [14]:
# import gspread
# import pandas as pd
# from oauth2client.service_account import ServiceAccountCredentials
# import json
# from google.colab import userdata

# # Get the service account key from Colab Secrets
# service_account_info = json.loads(userdata.get('SERVICE_ACCOUNT_JSON'))

# # Authenticate Google Sheets API
# scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
# creds = ServiceAccountCredentials.from_json_keyfile_dict(service_account_info, scope)
# client = gspread.authorize(creds)

# # Open Google Sheet
# # Replace "Tên_Google_Sheet" with your Google Sheet name
# # Replace "Tên_Sheet" with your sheet name
# spreadsheet = client.open_by_url("https://docs.google.com/spreadsheets/d/1RNrJ5B3pzLGNmxir2EguHH55vGUNHAOzaOV6Ng6_fVI/edit?gid=843162208#gid=843162208")
# sheet = spreadsheet.worksheet("Jobs 2025")
# data = sheet.get_all_records()
# # Convert to DataFrame
# df = pd.DataFrame(data)

In [15]:
def assign_region(places_str):
    if pd.isna(places_str):
        return 'Không rõ'

    places = [p.strip() for p in places_str.split(',') if p.strip()]

    # Từ khoá miền
    north_keywords = ['Hà Nội', 'Hải Phòng', 'Quảng Ninh', 'Bắc Ninh', 'Hưng Yên', 'Hải Dương',
                      'Thái Nguyên', 'Vĩnh Phúc', 'Phú Thọ', 'Bắc Giang', 'Thái Bình', 'Nam Định',
                      'Hà Nam', 'Ninh Bình', 'Hòa Bình', 'Sơn La', 'Lai Châu', 'Điện Biên',
                      'Lào Cai', 'Yên Bái', 'Tuyên Quang', 'Cao Bằng', 'Bắc Kạn', 'Lạng Sơn',
                      'Quảng Trị', 'Thừa Thiên Huế']

    central_keywords = ['Đà Nẵng', 'Quảng Nam', 'Quảng Ngãi', 'Bình Định', 'Phú Yên',
                        'Khánh Hòa', 'Ninh Thuận', 'Bình Thuận', 'Thanh Hóa', 'Nghệ An',
                        'Hà Tĩnh', 'Quảng Bình', 'Gia Lai', 'Kon Tum', 'Đắk Lắk',
                        'Đắk Nông', 'Lâm Đồng']

    south_keywords = ['Hồ Chí Minh', 'Sài Gòn', 'TP.HCM', 'Bình Dương', 'Đồng Nai', 'Bà Rịa - Vũng Tàu',
                      'Tây Ninh', 'Bình Phước']

    south_west_keywords = ['Long An', 'Tiền Giang', 'Bến Tre', 'Trà Vinh', 'Vĩnh Long',
                           'Đồng Tháp', 'An Giang', 'Kiên Giang', 'Cần Thơ', 'Hậu Giang',
                           'Sóc Trăng', 'Bạc Liêu', 'Cà Mau']

    regions = []

    for place in places:
        found = False

        for keyword in north_keywords:
            if keyword.lower() in place.lower():
                regions.append('Miền Bắc')
                found = True
                break
        if found: continue

        for keyword in central_keywords:
            if keyword.lower() in place.lower():
                regions.append('Miền Trung')
                found = True
                break
        if found: continue

        for keyword in south_west_keywords:
            if keyword.lower() in place.lower():
                regions.append('Miền Tây')
                found = True
                break
        if found: continue

        for keyword in south_keywords:
            if keyword.lower() in place.lower():
                regions.append('Miền Nam')
                found = True
                break
        # ⚠️ Không còn append 'Không rõ' ở đây

    # Loại trùng & loại 'Không rõ' nếu có sót
    unique_regions = list(set(regions))

    if len(places) == 1:
        return places[0]
    elif len(unique_regions) == 0:
        return 'Không rõ'
    elif len(unique_regions) == 1:
        return unique_regions[0]
    elif len(unique_regions) == 2:
        return ', '.join(sorted(unique_regions))
    elif len(unique_regions) >= 3:
        return 'Toàn quốc'
    else:
        return 'Không rõ'
df['Region'] = df['Places'].apply(assign_region)

In [16]:
# Xuất DataFrame df sang Google Sheets
spreadsheet = client.open_by_url("https://docs.google.com/spreadsheets/d/1E5EkOL7EJWQloDixisDQBN0mlVMBu4WC3MBgp4ze6cc/edit?gid=612484126#gid=612484126")
worksheet = spreadsheet.worksheet("Jobs 2025")      # tab đầu tiên
worksheet.clear()                                # xoá dữ liệu cũ
worksheet.update([df.columns.values.tolist()] + df.values.tolist())

print("✅ Đã ghi dữ liệu thành công lên Google Sheets!")


✅ Đã ghi dữ liệu thành công lên Google Sheets!
